### eSIM profiles

In [1]:
import pandas as pd
from pandas import Series, DataFrame
import datetime as dt
from os.path import join

In [3]:
downloads = r'c:\Users\balob\Documents\GITLAB\RB_BD\DATA\eSimProfiles\data'

if_keys = 'esim_rb_5k_18102019.out'

df_qr = pd.read_csv(join(downloads, 'RB_ICCID_QRCode_URL.csv'))
quantity = len(df_qr)

df_qr['ICCID_CD'] = df_qr.ICCID.str.slice(1,20)
df_qr.sort_values(by='ICCID_CD', inplace=True)
df_qr.reset_index(inplace=True, drop=True)

display(df_qr.head(3))

,ICCID,QR_Code,ICCID_CD
0,'89972191210000870004,LPA:1$rsp.truphone.com$4K-51Q6Z-LEOFIF,8997219121000087000
1,'89972191210000870012,LPA:1$rsp.truphone.com$4K-51Q76-1OTW1RH,8997219121000087001
2,'89972191210000870020,LPA:1$rsp.truphone.com$4K-51Q7D-89EUBX,8997219121000087002


In [4]:
# Из базы данных OCS выгружаем IMSI
#----------------------------------

import pandas as pd
import pyodbc 
#Connect to OCSDBREP1 (BSS)
cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER=172.18.11.82;DATABASE=10028;UID=BSS;PWD=iKQVm40AZAmyRaw72LeY')

sql_srt=\
'''
SELECT DISTINCT
r.RESELLER_NAME
,a.ACCOUNT_NAME
,ss.STATUS AS SUB_STATUS
,si.IMSI
,si.ICCID AS ICCID_CD
FROM RESELLERS r
INNER JOIN ACCOUNTS a
ON r.RESELLER_ID = a.RESELLER_ID
INNER JOIN SUBSCRIBERS sr
ON sr.ACCOUNT_ID = a.ACCOUNT_ID
INNER JOIN SUBSCRIBER_STATUS ss
ON ss.SUBSCRIBER_ID = sr.SUBSCRIBER_ID
INNER JOIN SUBSCRIBER_IMSIS si
ON ss.SUBSCRIBER_ID = si.SUBSCRIBER_ID
WHERE
r.deleted is NULL
AND ss.END_DATE IS NULL
AND ss.status = 'Active'
AND
(si.IMSI BETWEEN '260060145010000' AND '260060145014999')
'''

df_ocs_imsi = pd.read_sql_query(sql_srt, cnxn, coerce_float=False)
df_ocs_imsi['IMSI'] = df_ocs_imsi['IMSI'].astype('int64')
cnxn.close()
df_ocs_imsi.head(3)

,RESELLER_NAME,ACCOUNT_NAME,SUB_STATUS,IMSI,ICCID_CD
0,Flexiroam Limited -- TESTING at 2018-07,Test eSIM,Active,260060145014003,8997219121000091003
1,Flexiroam Limited -- TESTING at 2018-07,Test eSIM,Active,260060145014004,8997219121000091004
2,Flexiroam Limited -- TESTING at 2018-07,Test eSIM,Active,260060145014005,8997219121000091005


In [5]:
cols = ['ICCID','QR_Code','RESELLER_NAME','ACCOUNT_NAME', 'IMSI', 'SUB_STATUS']
df_batch = pd.merge(df_qr, df_ocs_imsi, on='ICCID_CD', how='left', suffixes=['_S2', '_S1'])

df_batch = df_batch[cols]
df_batch.head()

,ICCID,QR_Code,RESELLER_NAME,ACCOUNT_NAME,IMSI,SUB_STATUS
0,'89972191210000870004,LPA:1$rsp.truphone.com$4K-51Q6Z-LEOFIF,WMB Limited RT,WMB eSIMs,260060145010000,Active
1,'89972191210000870012,LPA:1$rsp.truphone.com$4K-51Q76-1OTW1RH,WMB Limited RT,WMB eSIMs,260060145010001,Active
2,'89972191210000870020,LPA:1$rsp.truphone.com$4K-51Q7D-89EUBX,WMB Limited RT,WMB eSIMs,260060145010002,Active
3,'89972191210000870038,LPA:1$rsp.truphone.com$4K-51Q9W-DK2E9Q,WMB Limited RT,WMB eSIMs,260060145010003,Active
4,'89972191210000870046,LPA:1$rsp.truphone.com$4K-51Q7K-10DA1Y5,WMB Limited RT,WMB eSIMs,260060145010004,Active


In [6]:
list_agg = [('IMSI_MIN','min'),('IMSI_MAX','max'),('NUM','count')]
list_col = ['RESELLER_NAME', 'ACCOUNT_NAME', 'SUB_STATUS', df_batch.IMSI.astype('str').str.slice(0,12)]
df_batch_group = df_batch.groupby(list_col)['IMSI'].agg(list_agg).reset_index().sort_values(by='IMSI')
df_batch_group.sort_values('IMSI_MIN')

,RESELLER_NAME,ACCOUNT_NAME,SUB_STATUS,IMSI,IMSI_MIN,IMSI_MAX,NUM
8,WMB Limited RT,WMB eSIMs,Active,260060145010,260060145010000,260060145010999,1000
7,STI - Telinta,G1S_eSIM,Active,260060145011,260060145011000,260060145011999,1000
1,Roamability Invertory,Roamability eSIM Inventory,Active,260060145012,260060145012000,260060145012999,1000
2,Roamability Invertory,Roamability eSIM Inventory,Active,260060145013,260060145013000,260060145013998,999
5,Roamability Test RT,Roamability Test Sim-cards,Active,260060145013,260060145013999,260060145013999,1
6,Roamability Test RT,Roamability Test Sim-cards,Active,260060145014,260060145014000,260060145014002,3
0,Flexiroam Limited -- TESTING at 2018-07,Test eSIM,Active,260060145014,260060145014003,260060145014007,5
4,Roamability Resellers Test,Vision,Active,260060145014,260060145014008,260060145014008,1
3,Roamability Invertory,Roamability eSIM Inventory,Active,260060145014,260060145014009,260060145014999,991


In [7]:
# Prepare the output file

qustomer = 'SimTex'
imsi_start = 260060145012000
given_quantity = 1000

current_date = dt.datetime.now().strftime('%y%m%d')

of_name = f'RB_eSIM_{qustomer}_{current_date}_{given_quantity}.csv'
df_customer = df_batch.loc[df_batch.IMSI.isin(range(imsi_start, imsi_start + given_quantity)),
       ['ICCID','QR_Code','IMSI']]
df_customer.to_csv(join(downloads, of_name), index=False)
df_customer.head()

,ICCID,QR_Code,IMSI
2000,'89972191210000890002,LPA:1$rsp.truphone.com$4K-524V3-NUT54C,260060145012000
2001,'89972191210000890010,LPA:1$rsp.truphone.com$4K-524VA-173WR3M,260060145012001
2002,'89972191210000890028,LPA:1$rsp.truphone.com$4K-524VO-IBFYCI,260060145012002
2003,'89972191210000890036,LPA:1$rsp.truphone.com$4K-524VV-IL5ILG,260060145012003
2004,'89972191210000890044,LPA:1$rsp.truphone.com$4K-524W2-EJJW7K,260060145012004
